In [1]:
import numpy as np
import math

In [2]:
class DataSplitByEntropy:
    def __init__(self, group, threshold):
        # 最大分组数
        self.max_group = group
        # 停止划分的最小熵
        self.min_threshold = threshold
        # 保存最终结果变量
        self.result = dict()
      
    # 加载数据，可以改写成读取外源数据，并整理成（特征，标签）的形式
    def load_data(self):
        data = np.array(
            [
                [56, 1],   [87, 1],  [129, 0],  [23, 0],   [342, 1], 
                [641, 1],  [63, 0],  [2764, 1], [2323, 0], [453, 1], 
                [10, 1],   [9, 0],   [88, 1],   [222, 0],  [97, 0],
                [2398, 1], [592, 1], [561, 1],  [764, 0],  [121, 1],
                [28, 0],   [49, 1],  [361, 1],  [164, 0],  [1210, 1]
            ]
        )
        return data

    # 计算信息熵
    def cal_entropy(self, data):
        num_data = len(data)
        label_counts = {}
        for feature in data:
            # 获得标签
            one_label = feature[-1]
            # 如果标签不在新定义的字典里则创建该标签
            label_counts.setdefault(one_label, 0) 
            # 该类标签下含有数据的个数 
            label_counts[one_label] += 1
        # 计算数据集整体的信息熵
        entropy = 0.0
        for key in label_counts:
            # 同类标签出现的概率
            prob = float(label_counts[key]) / num_data
            # 以2为底求对数
            entropy -= prob * math.log(prob, 2)
        return entropy

    # 按照调和信息熵最小化原则分割数据集
    def split(self, data):
        # inf为正无穷大
        min_entropy = np.inf
        # 记录最终分割索引,记录切割点
        index = -1
        # 按照第一列对数据进行排序
        sort_data = data[np.argsort(data[:, 0])]
        # 初始化最终分割数据后的熵
        last_e1, last_e2 = -1, -1
        # 返回的数据结构，包含数据和对应的熵
        S1 = dict()
        S2 = dict()
        for i in range(len(sort_data)):
            # 分割数据集
            split_data_1, split_data_2 = sort_data[: i + 1], sort_data[i + 1 :] 
            entropy1, entropy2 = ( self.cal_entropy(split_data_1), self.cal_entropy(split_data_2) ) # 计算信息熵
            entropy = entropy1 * len(split_data_1) / len(sort_data) +  entropy2 * len( split_data_2) / len(sort_data)
            # 如果加权熵小于最小值
            if entropy < min_entropy:
                min_entropy = entropy 
                index = i
                last_e1 = entropy1 
                last_e2 = entropy2
        S1["entropy"] = last_e1
        S1["data"] = sort_data[: index + 1]
        S2["entropy"] = last_e2
        S2["data"] = sort_data[index + 1 :]
        return S1, S2, entropy
    
    # 对数据进行分组
    def train(self, data):
        # 需要遍历的 key
        need_split_key = [0]
        # 将整个数据作为一组
        self.result.setdefault(0, {}) 
        self.result[0]["entropy"] = np.inf 
        self.result[0]["data"] = data
        group = 1
        for key in need_split_key:
            S1, S2, entropy = self.split(self.result[key]["data"])
            # 如果满足条件
            if entropy > self.min_threshold or group < self.max_group:
                self.result[key] = S1
                print(key)
                newKey = max(self.result.keys()) + 1 
                print(newKey)
                self.result[newKey] = S2
                print(self.result[newKey])
                need_split_key.extend([key]) 
                need_split_key.extend([newKey])
                print(need_split_key)
                group += 1
            else: 
                break                

In [3]:
if __name__ == "__main__":
    dse = DataSplitByEntropy(group=6,threshold=0.3) 
    data = dse.load_data()
    dse.train(data)
    print("result is {}".format(dse.result))

0
1
{'entropy': 0.6840384356390417, 'data': array([[ 342,    1],
       [ 361,    1],
       [ 453,    1],
       [ 561,    1],
       [ 592,    1],
       [ 641,    1],
       [ 764,    0],
       [1210,    1],
       [2323,    0],
       [2398,    1],
       [2764,    1]])}
[0, 0, 1]
0
2
{'entropy': 0.0, 'data': array([[129,   0],
       [164,   0],
       [222,   0]])}
[0, 0, 1, 0, 2]
1
3
{'entropy': 0.9709505944546686, 'data': array([[ 764,    0],
       [1210,    1],
       [2323,    0],
       [2398,    1],
       [2764,    1]])}
[0, 0, 1, 0, 2, 1, 3]
0
4
{'entropy': 0.863120568566631, 'data': array([[ 49,   1],
       [ 56,   1],
       [ 63,   0],
       [ 87,   1],
       [ 88,   1],
       [ 97,   0],
       [121,   1]])}
[0, 0, 1, 0, 2, 1, 3, 0, 4]
2
5
{'entropy': 0.0, 'data': array([[164,   0],
       [222,   0]])}
[0, 0, 1, 0, 2, 1, 3, 0, 4, 2, 5]
result is {0: {'entropy': 0.8112781244591328, 'data': array([[ 9,  0],
       [10,  1],
       [23,  0],
       [28,  0]])}, 1:

In [4]:
import numpy as np
data =np.array(
            [
                [56, 1],   [87, 1],  [129, 0],  [23, 0],   [342, 1], 
                [641, 1],  [63, 0],  [2764, 1], [2323, 0], [453, 1], 
                [10, 1],   [9, 0],   [88, 1],   [222, 0],  [97, 0],
                [2398, 1], [592, 1], [561, 1],  [764, 0],  [121, 1],
                [28, 0],   [49, 1],  [361, 1],  [164, 0],  [1210, 1]
            ]
        )

In [5]:
data

array([[  56,    1],
       [  87,    1],
       [ 129,    0],
       [  23,    0],
       [ 342,    1],
       [ 641,    1],
       [  63,    0],
       [2764,    1],
       [2323,    0],
       [ 453,    1],
       [  10,    1],
       [   9,    0],
       [  88,    1],
       [ 222,    0],
       [  97,    0],
       [2398,    1],
       [ 592,    1],
       [ 561,    1],
       [ 764,    0],
       [ 121,    1],
       [  28,    0],
       [  49,    1],
       [ 361,    1],
       [ 164,    0],
       [1210,    1]])

In [6]:
data[np.argsort(data[:, 0])]

array([[   9,    0],
       [  10,    1],
       [  23,    0],
       [  28,    0],
       [  49,    1],
       [  56,    1],
       [  63,    0],
       [  87,    1],
       [  88,    1],
       [  97,    0],
       [ 121,    1],
       [ 129,    0],
       [ 164,    0],
       [ 222,    0],
       [ 342,    1],
       [ 361,    1],
       [ 453,    1],
       [ 561,    1],
       [ 592,    1],
       [ 641,    1],
       [ 764,    0],
       [1210,    1],
       [2323,    0],
       [2398,    1],
       [2764,    1]])

In [7]:
label_counts={}
for feature in data:
            # 获得标签
    one_label = feature[-1]
            # 如果标签不在新定义的字典里则创建该标签
    label_counts.setdefault(one_label, 0) 
            # 该类标签下含有数据的个数 
    label_counts[one_label] += 1

In [8]:
label_counts

{1: 15, 0: 10}

In [9]:
result = {}
result.setdefault(0,{})

{}

In [10]:
result[0]["entropy"] = np.inf
result[0]["data"] = data
result

{0: {'entropy': inf,
  'data': array([[  56,    1],
         [  87,    1],
         [ 129,    0],
         [  23,    0],
         [ 342,    1],
         [ 641,    1],
         [  63,    0],
         [2764,    1],
         [2323,    0],
         [ 453,    1],
         [  10,    1],
         [   9,    0],
         [  88,    1],
         [ 222,    0],
         [  97,    0],
         [2398,    1],
         [ 592,    1],
         [ 561,    1],
         [ 764,    0],
         [ 121,    1],
         [  28,    0],
         [  49,    1],
         [ 361,    1],
         [ 164,    0],
         [1210,    1]])}}

In [11]:
import math

In [12]:
def cal_entropy(data):
    num_data = len(data)
    label_counts = {}
    for feature in data:
        # 获得标签
        one_label = feature[-1]
        # 如果标签不在新定义的字典里则创建该标签
        label_counts.setdefault(one_label, 0) 
        # 该类标签下含有数据的个数 
        label_counts[one_label] += 1
    # 计算数据集整体的信息熵
    entropy = 0.0
    for key in label_counts:
        # 同类标签出现的概率
        prob = float(label_counts[key]) / num_data
        # 以2为底求对数
        entropy -= prob * math.log(prob, 2)
    return entropy

In [13]:
def split(data):
    # inf为正无穷大
    min_entropy = np.inf
    # 记录最终分割索引
    index = -1
    # 按照第一列对数据进行排序
    sort_data = data[np.argsort(data[:, 0])]
    # 初始化最终分割数据后的熵
    last_e1, last_e2 = -1, -1
    # 返回的数据结构，包含数据和对应的熵
    S1 = dict()
    S2 = dict()
    for i in range(len(sort_data)):
        # 分割数据集
        split_data_1, split_data_2 = sort_data[: i + 1], sort_data[i + 1 :] 
        entropy1, entropy2 = (cal_entropy(split_data_1), cal_entropy(split_data_2) ) # 计算信息熵
        entropy = entropy1 * len(split_data_1) / len(sort_data) +  entropy2 * len( split_data_2) / len(sort_data)
        # 如果加权平均熵小于最小值
        if entropy < min_entropy:
            min_entropy = entropy 
            index = i
            last_e1 = entropy1 
            last_e2 = entropy2
    S1["entropy"] = last_e1
    S1["data"] = sort_data[: index + 1]
    S2["entropy"] = last_e2
    S2["data"] = sort_data[index + 1 :]
    return S1, S2, entropy

In [14]:
S1, S2, entropy = split(result[0]["data"])

In [15]:
S1, S2

({'entropy': 0.9852281360342516,
  'data': array([[  9,   0],
         [ 10,   1],
         [ 23,   0],
         [ 28,   0],
         [ 49,   1],
         [ 56,   1],
         [ 63,   0],
         [ 87,   1],
         [ 88,   1],
         [ 97,   0],
         [121,   1],
         [129,   0],
         [164,   0],
         [222,   0]])},
 {'entropy': 0.6840384356390417,
  'data': array([[ 342,    1],
         [ 361,    1],
         [ 453,    1],
         [ 561,    1],
         [ 592,    1],
         [ 641,    1],
         [ 764,    0],
         [1210,    1],
         [2323,    0],
         [2398,    1],
         [2764,    1]])})

In [16]:
need_split_key = [0]

In [17]:
result[0] = S1
newKey = max(result.keys()) + 1 
result[newKey] = S2
need_split_key.extend([0]) 
need_split_key.extend([newKey])

In [18]:
result

{0: {'entropy': 0.9852281360342516,
  'data': array([[  9,   0],
         [ 10,   1],
         [ 23,   0],
         [ 28,   0],
         [ 49,   1],
         [ 56,   1],
         [ 63,   0],
         [ 87,   1],
         [ 88,   1],
         [ 97,   0],
         [121,   1],
         [129,   0],
         [164,   0],
         [222,   0]])},
 1: {'entropy': 0.6840384356390417,
  'data': array([[ 342,    1],
         [ 361,    1],
         [ 453,    1],
         [ 561,    1],
         [ 592,    1],
         [ 641,    1],
         [ 764,    0],
         [1210,    1],
         [2323,    0],
         [2398,    1],
         [2764,    1]])}}

In [19]:
need_split_key

[0, 0, 1]